<a href="https://colab.research.google.com/github/Deepsphere-AI/DSAI_Python_Programming/blob/main/Unit-16/Python%20for%20Deep%20Learning/Program%20342%20-%20CSLAB_DEEP_LEARNING_NORMALIZING_CREATING_SEQUENCES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

# /*********************************************************************************************************************
  
#   File Name 	:   CSLAB_DEEP_LEARNING_NORMALIZING_CREATING_SEQUENCES
#   Purpose 	:   A Program in Python for Normalizing & Creating Sequences using Deep Learning
#   Author	:   Deepsphere.ai
#   Reviewer 	:   Jothi Periasamy
#   Date :   26/10/2022
#   Version	:   1.0	

# /***********************************************************************************************************************

## Program Description : Program for Normalizing & Creating Sequences using Deep Learning in Python

## Python Development Environment & Runtime - Python, Anaconda

import pandas as vAR_pd

import pandas as vAR_pd

from google.colab import drive

drive.mount('/content/drive')

vAR_CSLAB_CREATE_SEQUENCES = '/content/drive/MyDrive/Semester-4/Deep Learning for Enterprise/Data/crypto_data/LTC-USD.csv'

vAR_CSLAB_df = vAR_pd.read_csv(vAR_CSLAB_CREATE_SEQUENCES, names=['time', 'low', 'high', 'open', 'close', 'volume'])

print(vAR_CSLAB_df.head())

vAR_CSLAB_main_df = vAR_pd.DataFrame() # begin empty

ratios = ["BTC-USD", "LTC-USD", "BCH-USD", "ETH-USD"]  # the 4 ratios we want to consider
for ratio in ratios:  # begin iteration
    print(ratio)
    vAR_CSLAB_dataset = "/content/drive/MyDrive/Semester-4/Deep Learning for Enterprise/Data/crypto_data/BTC-USD.csv"  # get the full path to the file.
    vAR_CSLAB_df = vAR_pd.read_csv(vAR_CSLAB_dataset, names=['time', 'low', 'high', 'open', 'close', 'volume'])  # read in specific file

    # rename volume and close to include the ticker so we can still which close/volume is which:
    vAR_CSLAB_df.rename(columns={"close": f"{ratio}_close", "volume": f"{ratio}_volume"}, inplace=True)

    vAR_CSLAB_df.set_index("time", inplace=True)  # set time as index so we can join them on this shared time
    vAR_CSLAB_df = vAR_CSLAB_df[[f"{ratio}_close", f"{ratio}_volume"]]  # ignore the other columns besides price and volume

    if len(vAR_CSLAB_main_df)==0:  # if the dataframe is empty
        vAR_CSLAB_main_df = vAR_CSLAB_df  # then it's just the current df
    else:  # otherwise, join this data to the main one
        vAR_CSLAB_main_df = vAR_CSLAB_main_df.join(vAR_CSLAB_df)
        
vAR_CSLAB_main_df.fillna(method="ffill", inplace=True)  # if there are gaps in data, use previously known values
vAR_CSLAB_main_df.dropna(inplace=True)
print(vAR_CSLAB_main_df.head())  # how did we do??

vAR_CSLAB_SEQ_LEN = 60  # how long of a preceeding sequence to collect for RNN
vAR_CSLAB_FUTURE_PERIOD_PREDICT = 3  # how far into the future are we trying to predict?
vAR_CSLAB_RATIO_TO_PREDICT = "LTC-USD"

def classify(current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0
    
vAR_CSLAB_main_df['future'] = vAR_CSLAB_main_df[f'{vAR_CSLAB_RATIO_TO_PREDICT}_close'].shift(-vAR_CSLAB_FUTURE_PERIOD_PREDICT)

vAR_CSLAB_main_df['target'] = list(map(classify, vAR_CSLAB_main_df[f'{vAR_CSLAB_RATIO_TO_PREDICT}_close'], vAR_CSLAB_main_df['future']))

print(vAR_CSLAB_main_df.head())

# /****************************************************************************************************************************
#   Disclaimer.

# We are providing this code block strictly for learning and researching, this is not a production
# ready code. We have no liability on this particular code under any circumstances; users should use
# this code on their own risk. All software, hardware and othr products that are referenced in these 
# materials belong to the respective vendor who developed or who owns this product.

# /****************************************************************************************************************************
  

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
         time        low       high       open      close      volume
0  1528968660  96.580002  96.589996  96.589996  96.580002    9.647200
1  1528968720  96.449997  96.669998  96.589996  96.660004  314.387024
2  1528968780  96.470001  96.570000  96.570000  96.570000   77.129799
3  1528968840  96.449997  96.570000  96.570000  96.500000    7.216067
4  1528968900  96.279999  96.540001  96.500000  96.389999  524.539978
BTC-USD
LTC-USD
BCH-USD
ETH-USD
            BTC-USD_close  BTC-USD_volume  LTC-USD_close  LTC-USD_volume  \
time                                                                       
1528968660    6489.549805        0.587100    6489.549805        0.587100   
1528968720    6487.379883        7.706374    6487.379883        7.706374   
1528968780    6479.410156        3.088252    6479.410156        3.088252   
1528968840    6479.410156        1.4041